In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
store_100_path = r"df_all_store_100.parquet"
store_42_path = r"df_all_store_42.parquet"
store_4906_path = r"df_all_store_4906.parquet"

In [10]:
df_100 = pd.read_parquet(store_100_path)
df_42 = pd.read_parquet(store_42_path)
df_4906 = pd.read_parquet(store_4906_path)
df_all = pd.concat([df_100, df_42, df_4906])

In [11]:
del df_100, df_42, df_4906

In [12]:
df_all['date'] = pd.to_datetime(df_all['date'])

In [13]:
warehouse_path = r"df_all_store_VZ01.parquet"
df_warehouse = pd.read_parquet(warehouse_path)

In [14]:
df_warehouse = df_warehouse.rename(columns={"stock": "warehouse_stock"})
df_warehouse['date'] = pd.to_datetime(df_warehouse['date'])

In [15]:
df_warehouse['sku'] = df_warehouse['sku'].astype(str)


In [16]:
df_warehouse = df_warehouse[df_warehouse['date'] > '2018-01-01']

In [17]:
df_all = df_all.merge(df_warehouse[["sku","date","warehouse_stock"]], on = ["sku","date"], how = "left")


In [18]:
df_all = df_all[df_all['date'] > '2018-01-01']

In [19]:
df_all = df_all.sort_values('date')

In [20]:
df_all = df_all.merge(df_all[df_all['stock'] > 0].groupby(['sku', 'store']).agg({'stock': 'first'}).reset_index().rename(columns={'stock':'palmers_first_allocation'}), on = ['sku', 'store'], how = 'left')

In [21]:
sku_store_df_step_g = df_all.groupby(['sku', 'store'])

In [22]:
import json
dict_per_store_per_year_path = r"dict_per_store_per_year.json"
with open(dict_per_store_per_year_path) as json_file:
    dict_per_store_per_year = json.load(json_file)

In [23]:
first_dist_store_100 =  dict_per_store_per_year['2020']['100'] / (dict_per_store_per_year['2020']['100'] + dict_per_store_per_year['2020']['42'])
first_dist_store_42 =  dict_per_store_per_year['2020']['42'] / (dict_per_store_per_year['2020']['100'] + dict_per_store_per_year['2020']['42'])

In [24]:
first_dist_store_100, first_dist_store_42

(0.6374031057660117, 0.3625968942339883)

In [25]:
store_100_dist_need_to_be = 51 / 104
store_100_dist_need_to_be

0.49038461538461536

In [26]:
store_42_dist_need_to_be = 53 / 104
store_42_dist_need_to_be

0.5096153846153846

In [94]:
i = 0

dict_of_all_data_after_preprocess = {}
for sku in ['100630105000002']:
    dict_of_all_data_after_preprocess[sku] = {}
    for store in ['42', '100']:
        dict_of_all_data_after_preprocess[sku][store] = {}

        sku_store_df_step = sku_store_df_step_g.get_group((sku, store))
        first_stock = sku_store_df_step[sku_store_df_step["stock"]>0]["stock"].iloc[0]
        # nan in our_stock if before first stock
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "palmers_first_allocation"] = np.nan
        sku_store_df_step['days_from_first_stock'] = 1
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "days_from_first_stock"] = np.nan
        sku_store_df_step['days_from_first_stock'] = sku_store_df_step['days_from_first_stock'].cumsum()
        sku_store_df_step['sales_cumsum'] = sku_store_df_step['sales'].cumsum()
        sku_store_df_step['dynamic_stock_with_palmers_init'] = sku_store_df_step['palmers_first_allocation'] - sku_store_df_step['sales_cumsum']
        sku_store_df_step['sales_cumsum_divide_days_from_first_stock'] =  sku_store_df_step['sales_cumsum'] / sku_store_df_step['days_from_first_stock']

        zero_sales_days_back = 1
        sku_store_df_step['days_from_last_sales_that_not_zero'] = 1
        for index, row in sku_store_df_step.shift(1).iterrows():
            # if index is sku_store_df_step[sku_store_df_step["stock"]>0]["stock"] index
            if index > sku_store_df_step[sku_store_df_step["stock"]>0].index[0]:
                if row['sales'] == 0:
                    zero_sales_days_back += 1
                else:
                    zero_sales_days_back = 1
                sku_store_df_step.at[index, 'days_from_last_sales_that_not_zero'] = zero_sales_days_back
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "days_from_last_sales_that_not_zero"] = np.nan
        sku_store_df_step['rate_sales'] = sku_store_df_step['sales'] / sku_store_df_step['days_from_last_sales_that_not_zero']
        dict_of_all_data_after_preprocess[sku][store] = sku_store_df_step
        sku_store_df_step['diff_stock'] = sku_store_df_step['stock'].diff()




/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_37550/2893759788.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sku_store_df_step['days_from_first_stock'] = 1
/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_37550/2893759788.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sku_store_df_step['days_from_first_stock'] = sku_store_df_step['days_from_first_stock'].cumsum()
/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_37550/2893759788.py:16: SettingWithCopyWarning: 
A 

In [76]:
print(sku_store_df_step.iloc[100:])

                      sku store       date  average_price  \
18058906  100630105000002   100 2020-09-07            NaN   
18068852  100630105000002   100 2020-09-08            NaN   
18109716  100630105000002   100 2020-09-09            NaN   
18153253  100630105000002   100 2020-09-10            NaN   
18164071  100630105000002   100 2020-09-11            NaN   
...                   ...   ...        ...            ...   
73205453  100630105000002   100 2023-12-08            NaN   
73248678  100630105000002   100 2023-12-09            NaN   
73300214  100630105000002   100 2023-12-10            NaN   
73355073  100630105000002   100 2023-12-11            NaN   
73388217  100630105000002   100 2023-12-12            NaN   

          average_dicounted_price  sales          item            sku_store  \
18058906                      NaN    0.0  100630105000  100630105000002,100   
18068852                      NaN    0.0  100630105000  100630105000002,100   
18109716                      

In [97]:

dynamic_distribution_per_store = {}
date = '2020-09-12'
total_stock_warehouse = 104
update_stock_remain = 104
warehouse_dict = {}
for sku in dict_of_all_data_after_preprocess.keys():
    dynamic_distribution_per_store[sku] = {}
    warehouse_dict[sku] = {}
    for store in dict_of_all_data_after_preprocess[sku].keys():
        dynamic_distribution_per_store[sku][store] = {}
        date_data = dict_of_all_data_after_preprocess[sku][store][dict_of_all_data_after_preprocess[sku][store]['date'] <= date]
        dynamic_distribution_per_store[sku][store]['initial_stock_in_store'] = date_data['palmers_first_allocation'].iloc[-1]
        dynamic_distribution_per_store[sku][store]['current_stock_in_store'] = date_data['dynamic_stock_with_palmers_init'].iloc[-1]
        dynamic_distribution_per_store[sku][store]['sales_cumsum'] =  date_data.set_index('date')['sales_cumsum']
        dynamic_distribution_per_store[sku][store]['rate_sales'] = date_data.set_index('date')['rate_sales']
        dynamic_distribution_per_store[sku][store]['sales_cumsum_divide_days_from_first_stock'] = date_data.set_index('date')['sales_cumsum_divide_days_from_first_stock']
        dynamic_distribution_per_store[sku][store]['initial_stock_in_warehouse'] = total_stock_warehouse
        update_stock_remain -= date_data['diff_stock'][date_data['diff_stock'] > 0].sum() - np.abs(date_data['sales'][date_data['sales'] < 0].sum())
        dynamic_distribution_per_store[sku][store]['prior_original_distribution'] = dict_per_store_per_year['2020'][store] / (dict_per_store_per_year['2020']['100'] + dict_per_store_per_year['2020']['42'])
        dynamic_distribution_per_store[sku][store]['prior_original_allocation_stock'] = np.round(dynamic_distribution_per_store[sku][store]['prior_original_distribution'] * total_stock_warehouse)

        dynamic_distribution_per_store[sku][store]['dynamic_distribution'] =  dynamic_distribution_per_store[sku][store]['prior_original_distribution']
        dynamic_distribution_per_store[sku][store]['dynamic_allocation_stock'] = dynamic_distribution_per_store[sku][store]['prior_original_allocation_stock']

        dynamic_distribution_per_store[sku][store]['action'] = 0

    warehouse_dict[sku]['initial_stock_in_warehouse'] = total_stock_warehouse
    warehouse_dict[sku]['remain_stock'] = update_stock_remain
    for store in dict_of_all_data_after_preprocess[sku].keys():
        dynamic_distribution_per_store[sku][store]['initial_stock_in_warehouse'] = warehouse_dict[sku]['initial_stock_in_warehouse']
        dynamic_distribution_per_store[sku][store]['remain_stock'] = warehouse_dict[sku]['remain_stock']



In [98]:
dynamic_distribution_per_store

{'100630105000002': {'42': {'initial_stock_in_store': 20.0,
   'current_stock_in_store': 20.0,
   'sales_cumsum': date
   2020-09-02    0.0
   2020-09-03    0.0
   2020-09-04    0.0
   2020-09-05    0.0
   2020-09-06    0.0
   2020-09-07    0.0
   2020-09-08    0.0
   2020-09-09    0.0
   2020-09-10    0.0
   2020-09-11    0.0
   2020-09-12    0.0
   Name: sales_cumsum, dtype: float64,
   'rate_sales': date
   2020-09-02    NaN
   2020-09-03    NaN
   2020-09-04    NaN
   2020-09-05    NaN
   2020-09-06    NaN
   2020-09-07    NaN
   2020-09-08    NaN
   2020-09-09    NaN
   2020-09-10    NaN
   2020-09-11    0.0
   2020-09-12    0.0
   Name: rate_sales, dtype: float64,
   'sales_cumsum_divide_days_from_first_stock': date
   2020-09-02    NaN
   2020-09-03    NaN
   2020-09-04    NaN
   2020-09-05    NaN
   2020-09-06    NaN
   2020-09-07    NaN
   2020-09-08    NaN
   2020-09-09    NaN
   2020-09-10    NaN
   2020-09-11    0.0
   2020-09-12    0.0
   Name: sales_cumsum_divide_days_fro

In [90]:
warehouse_dict

{'100630105000002': {'initial_stock_in_warehouse': 104, 'remain_stock': 59.0}}

In [ ]:
def update_dynamic_distribution(store_data, total_stock_warehouse):
    # Recalculate the distribution ratio based on new sales data and other metrics
    # This could involve complex business logic depending on your specific needs
    # For example, adjust the distribution based on the rate of sales
    if store_data['action'] == 'increase_stock':
        # Increase the distribution ratio for this store
        store_data['dynamic_distribution_before_norm'] += 0.02  # Define how you calculate this
    elif store_data['action'] == 'decrease_stock':
        # Decrease the distribution ratio for this store
        store_data['dynamic_distribution_before_norm'] -= 0.02
    else:

    # store_data['dynamic_distribution'] = max(0, min(store_data['dynamic_distribution'], 1))
    # # Recalculate the dynamic allocation of stock based on the updated distribution
    # store_data['dynamic_allocation_stock'] = round(store_data['dynamic_distribution'] * total_stock_warehouse)
    # # Adjust the remaining stock in the warehouse
    # store_data['remain_stock'] -= store_data['next_amount_allocation']
    # store_data['remain_stock'] = max(0, store_data['remain_stock'])  # Ensure it doesn't go negative
    # # Update the prioritization of the original distribution if needed


In [ ]:
def compare_and_update_belief(dynamic_distribution, deterministic_sales_percentage, current_date):
    redistribution_suggestions = {}
    for sku, stores in dynamic_distribution.items():
        for store, data in stores.items():
            # Extract necessary data
            expected_sales = deterministic_sales_percentage[sku][store] * data['initial_stock']
            actual_sales = data['sales_cumsum'][-1]
            rate_sales = data['rate_sales']
            sales_cumsum_div_days = data['sales_cumsum_divide_days_from_first_stock']
            # Analyzing Sales Performance
            if actual_sales < expected_sales:
                if rate_sales < some_rate_threshold and sales_cumsum_div_days < some_cumsum_div_days_threshold:
                    data['action'] = 'decrease_stock'
                    redistribution_suggestions.setdefault(sku, []).append((store, 'decrease'))
                else:
                    data['action'] = 'monitor'
            else:
                if sales_cumsum_div_days > some_cumsum_div_days_threshold:
                    data['action'] = 'increase_stock'
                    redistribution_suggestions.setdefault(sku, []).append((store, 'increase'))
                else:
                    data['action'] = 'maintain_current_strategy'
            # Belief Update
            sales_difference = actual_sales - expected_sales
            belief_update = sales_difference / expected_sales if expected_sales != 0 else 0
            data['belief_update'] = belief_update
            # Call to update store's stock based on decided action
            update_store_stock(data, current_date)
    # Handling Redistribution
    handle_redistribution(redistribution_suggestions, dynamic_distribution)
def handle_redistribution(redistribution_suggestions, dynamic_distribution):
    # Logic to handle stock redistribution based on suggestions
    for sku, suggestions in redistribution_suggestions.items():
        decrease_stores = [store for store, action in suggestions if action == 'decrease']
        increase_stores = [store for store, action in suggestions if action == 'increase']
        # Logic to determine how much stock to move from decrease_stores to increase_stores
        # Example: Transfer stock based on the rate of sales, stock remain, etc.
        # Implement redistribution logic here
        pass
def update_store_stock(data, current_date):
    # Implement the logic for updating the stock based on the action
    # Example:
    if data['action'] == 'increase_stock':
        # Logic to increase stock
        pass
    elif data['action'] == 'consider_discount_or_redistribution':
        # Logic for discounts or redistribution
        pass
    # ... additional conditions based on the 'action'






In [28]:
print(sku_store_df_step.iloc[0:30])

                     sku store       date  average_price  \
2014680  100009161000001   100 2018-09-12            NaN   
2024532  100009161000001   100 2018-09-13            NaN   
2043874  100009161000001   100 2018-09-14            NaN   
2050162  100009161000001   100 2018-09-15            NaN   
2064786  100009161000001   100 2018-09-16            NaN   
2075427  100009161000001   100 2018-09-17            NaN   
2088404  100009161000001   100 2018-09-18            NaN   
2092978  100009161000001   100 2018-09-19            NaN   
2104613  100009161000001   100 2018-09-20            NaN   
2124014  100009161000001   100 2018-09-21            NaN   
2129414  100009161000001   100 2018-09-22            NaN   
2144075  100009161000001   100 2018-09-23            NaN   
2158493  100009161000001   100 2018-09-24            NaN   
2166636  100009161000001   100 2018-09-25            NaN   
2173706  100009161000001   100 2018-09-26            NaN   
2187204  100009161000001   100 2018-09-2

In [29]:
sku_store_df_step.loc[sku_store_df_step['date'] < sku_store_df_step[sku_store_df_step["stock"]>0]['date'].iloc[0], "sum_of_our_allocation_stock"] = np.nan

/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_37550/4001794443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sku_store_df_step.loc[sku_store_df_step['date'] < sku_store_df_step[sku_store_df_step["stock"]>0]['date'].iloc[0], "sum_of_our_allocation_stock"] = np.nan


In [30]:
sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]]

/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_37550/1012264236.py:1: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]]


TypeError: Invalid comparison between dtype=datetime64[ns] and ndarray

In [ ]:
print(df_all[(df_all['sku'] == '100009161000001') & (df_all['store'] == '100')])